In [1]:
import os
import ee
import geemap
import pandas as pd

In [2]:
data = pd.read_csv("GHG_Data_Sample.csv")

In [3]:
data['Date'] = pd.to_datetime(data['Date'], format = '%m/%d/%y')
data.head()

,Date,Site,ID,Lat,Long,CO2PixelDay
0,2015-07-15,Goodrich,R1,40.366021,-120.949558,80.613691
1,2015-07-15,Goodrich,R10,40.365042,-120.949802,82.194613
2,2015-07-15,Goodrich,R11,40.365238,-120.950044,44.048585
3,2015-07-15,Goodrich,R12,40.365436,-120.950289,94.443643
4,2015-07-15,Goodrich,R2,40.365824,-120.949316,157.806186


In [4]:
#ee.Authenticate()
ee.Initialize()

In [5]:
def maskClouds(image):
    quality = image.select('pixel_qa')
    cloud = quality.bitwiseAnd(1 << 5).eq(0)    # mask out cloud shadow
    clear = quality.bitwiseAnd(1 << 4).eq(0)     # mask out cloud
    return image.updateMask(cloud).updateMask(clear)

In [6]:
def getBandValues(landsat_collection, point, target_date, bufferDays = 30, landsatNo = 8):
    spatial_filtered = landsat_collection.filterBounds(point)
    temporal_filtered = spatial_filtered.filterDate(ee.Date(target_date).advance(-bufferDays, 'day'), ee.Date(target_date).advance(bufferDays, 'day'))
    cloud_free_images = temporal_filtered.map(maskClouds)
    sorted_collection = cloud_free_images.sort('cloud')
    image_list = sorted_collection.toList(sorted_collection.size())
    noImages = image_list.size().getInfo()
    nImage, band_values = 0, {'B2': None}
    
    while band_values['B2'] == None and nImage < noImages:
        nearest_image = ee.Image(image_list.get(nImage))
        nImage += 1
        if landsatNo == 7:
            bands = nearest_image.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
        else:
            bands = nearest_image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])
        band_values = bands.reduceRegion(ee.Reducer.mean(), point, 30).getInfo()
    
    return band_values

In [7]:
landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
landsat7_collection = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')

In [19]:
Blue, Green, Red, NIR, SWIR_1, SWIR_2 = [], [], [], [], [], []

for id in range(data.shape[0]):
    x, y = data.loc[id, ['Long', 'Lat']]
    point = ee.Geometry.Point(x, y)
    target_date = data.loc[id, 'Date'].strftime('%Y-%m-%d')
    band_values = getBandValues(landsat8_collection, point, target_date)
    if band_values['B2']:
        Blue.append(band_values['B2'])
        Green.append(band_values['B3'])
        Red.append(band_values['B4'])
        NIR.append(band_values['B5'])
        SWIR_1.append(band_values['B6'])
        SWIR_2.append(band_values['B7'])
    else:
        band_values = getBandValues(landsat7_collection, point, target_date, 30, 7)
        Blue.append(band_values['B1'])
        Green.append(band_values['B2'])
        Red.append(band_values['B3'])
        NIR.append(band_values['B4'])
        SWIR_1.append(band_values['B5'])
        SWIR_2.append(band_values['B7'])
    
    if id % 100 == 0:
        print(id, end = ' ')

data['Blue'] = Blue
data['Green'] = Green
data['Red'] = Red
data['NIR'] = NIR
data['SWIR_1'] = SWIR_1
data['SWIR_2'] = SWIR_2

data.head(10)

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 

,Date,Site,ID,Lat,Long,CO2PixelDay,Blue,Green,Red,NIR,SWIR_1,SWIR_2
0,2015-07-15,Goodrich,R1,40.366021,-120.949558,80.613691,512.0,775.0,951.0,2123.0,2965.0,2136.0
1,2015-07-15,Goodrich,R10,40.365042,-120.949802,82.194613,481.0,732.0,888.0,2035.0,2820.0,2097.0
2,2015-07-15,Goodrich,R11,40.365238,-120.950044,44.048585,482.0,730.0,860.0,2101.0,2727.0,1974.0
3,2015-07-15,Goodrich,R12,40.365436,-120.950289,94.443643,468.0,753.0,861.0,2394.0,2800.0,1850.0
4,2015-07-15,Goodrich,R2,40.365824,-120.949316,157.806186,512.0,775.0,951.0,2123.0,2965.0,2136.0
5,2015-07-15,Goodrich,R24,40.362695,-120.952175,24.530692,411.0,663.0,759.0,2272.0,2648.0,1697.0
6,2015-07-15,Goodrich,R3,40.365629,-120.949073,82.042008,501.0,741.0,924.0,1903.0,2959.0,2310.0
7,2015-07-15,Goodrich,R4,40.365433,-120.949317,74.781880,453.0,690.0,835.0,1984.0,2704.0,2032.0
8,2015-07-15,Goodrich,R5,40.365631,-120.949561,75.322777,495.0,761.0,919.0,2138.0,2959.0,2123.0
9,2015-07-15,Goodrich,R6,40.365827,-120.949802,102.295711,497.0,765.0,936.0,2235.0,2956.0,2065.0


In [20]:
len([x for x in Blue if x])

2744

In [21]:
data.to_csv('GHG_Data_Sample_Bands.csv', index=False)

In [14]:
x, y = data.loc[1, ['Long', 'Lat']]
point = ee.Geometry.Point(x, y)

In [15]:
target_date = data.loc[1, 'Date'].strftime('%Y-%m-%d')

In [ ]:
spatial_filtered = landsat_collection.filterBounds(point)
temporal_filtered = spatial_filtered.filterDate(ee.Date(target_date).advance(-7, 'day'), ee.Date(target_date).advance(7, 'day'))

In [ ]:
cloud_free_images = temporal_filtered.map(maskClouds)#.map(calculateCloudScore)

In [ ]:
sorted_collection = cloud_free_images.sort('cloud')

In [ ]:
nearest_image = ee.Image(sorted_collection.first())

In [ ]:
nearest_image

In [ ]:
bands = nearest_image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])

In [ ]:
bands

In [ ]:
band_values = bands.reduceRegion(ee.Reducer.mean(), point, 30).getInfo()

In [ ]:
print('Band Values:', band_values)

In [ ]:
map_l8 = geemap.Map(center=[y,x], zoom=10)
image_viz_params = {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}

# Add the image layer to the map and display it.
map_l8.add_layer(nearest_image, image_viz_params, 'false color composite')
display(map_l8)